In [1]:
import numpy as np

# Problem 1

In [19]:
def calc_freq(start_freq,input_file):
    freq = start_freq
    with open(input_file,'r') as f:
        for line in f:
            freq += int(line)
    return freq

def calc_freq_repeat(start_freq,input_file):
    freq = start_freq
    repeat = set()
    first_repeat = None
    flines = []
    
    with open(input_file,'r') as f:
        flines = [int(line) for line in f]
        
    while not first_repeat:
        for i in flines:
            freq += i
            if freq in repeat:
                first_repeat = freq
                break
            repeat.add(freq)
    print(first_repeat)
    return freq

In [20]:
calc_freq(0,'input1.txt')

547

In [21]:
calc_freq_repeat(0,'input1.txt')

76414


76414

# Problem 2

In [31]:
import numpy as np

In [11]:
def calc_checksum(name_list):
    twos = 0
    threes = 0
    for name in name_list:
        char_ct = {}
        for char in name:
            char_ct[char] = char_ct.get(char,0) + 1
        twos_check = (len([k for k,v in char_ct.items() if v==2]) > 0)*1
        threes_check = (len([k for k,v in char_ct.items() if v==3]) > 0)*1
        twos += twos_check
        threes += threes_check
    return twos*threes

In [ ]:
          if s[i] = t[j]:
            substitutionCost := 0
          else:
            substitutionCost := 1

          d[i, j] := minimum(d[i-1, j] + 1,                   // deletion
                             d[i, j-1] + 1,                   // insertion
                             d[i-1, j-1] + substitutionCost)  // substitution
 
  return d[m, n]

In [43]:
def lev_dist(s,t):
    m = len(s)
    n = len(t)
    
    d = np.zeros((m,n))
    d[:,0] = range(m)
    d[0,:] = range(n)
    
    for j in range(n):
        for i in range(m):
            if s[i] == t[j]:
                cost = 0
            else:
                cost = 1
            d[i,j] = min(d[i-1,j]+1,
                               d[i,j-1] + 1,
                               d[i-1,j-1] + cost)
        
    return d[m-1,n-1]

In [1]:
name_list = [
    'abcdef',
    'bababc',
    'abbcde',
    'abcccd',
    'aabcdd',
    'abcdee',
    'ababab'
]

In [13]:
calc_checksum(name_list)

12

In [17]:
with open('input2.txt','r') as f:
    name_list = f.readlines()
    name_list = [name.strip() for name in name_list]

In [19]:
calc_checksum(name_list)

5750

In [47]:
for s in name_list:
    for t in name_list:
        if lev_dist(s,t) == 1:
            print(s,t)
            break

tzyvunogzariwkpcbdewsmjhxi tzyvunogzariwkpcbdewmmjhxi


KeyboardInterrupt: 

# Problem 3

In [122]:
class Claim:
    def __init__(self,claim_str):
        # #123 @ 3,2: 5x4
        claim_list = claim_str.replace('@ ','').replace(':','').split(' ')
        
        claim_num = claim_list[0][1:]
        self.claim_num = claim_num
        
        xy = claim_list[1].split(',')
        self.x = int(xy[0])
        self.y = int(xy[1])
        wh = claim_list[2].split('x')
        self.width = int(wh[0])
        self.height = int(wh[1])
        
        self.max_x = self.x + self.width - 1
        self.max_y = self.y + self.height - 1
    
    def get_bounds(self):
        min_x = self.x
        min_y = self.y
        max_x = self.max_x
        max_y = self.max_y
        return (min_x,max_x,min_y,max_y)
    
    def contains_pt(self,pt_x,pt_y):
        in_x = pt_x >= self.x and pt_x <= self.max_x
        in_y = pt_y >= self.y and pt_y <= self.max_y
        
        return in_x and in_y
    
    def get_matrix(self,max_dim):
        full_mat = np.zeros((max_dim,max_dim))
        min_x = self.x
        min_y = self.y
        max_x = self.max_x + 1
        max_y = self.max_y + 1
        full_mat[min_x:max_x,min_y:max_y] = 1
        return full_mat
    
    def get_area(self):
        return self.width * self.height
    
def calculate_all_overlaps(claims):
    max_pt = 1000
    
    full_mat = np.zeros((max_pt,max_pt))
    for c in claims:
        full_mat = full_mat + c.get_matrix(max_pt)
    return full_mat

In [71]:
claim_str = '#123 @ 3,2: 5x4'
c = Claim(claim_str)

In [94]:
claim_strs = [
    '#1 @ 1,3: 4x4',
    '#2 @ 3,1: 4x4',
    '#3 @ 5,5: 2x2',
]
claims = [Claim(c) for c in claim_strs]

In [123]:
with open('input3.txt','r') as f:
    claim_list = f.readlines()
    claim_list = [Claim(c) for c in claim_list]

In [124]:
m = calculate_all_overlaps(claim_list)
print('Answer: %i'%(np.sum((m >= 2)*1)))

In [133]:
np.sum((m >= 2)*1)

107820